In [1]:
import json
import os
import pandas as pd

In [2]:
os.chdir('../')

In [3]:
from src.data.ClickhouseClient import ClickhouseClient

In [4]:
# Get Clickhouse credentials and connect
filename_credentials = 'clickhouse_credentials.json'
with open(filename_credentials, 'r') as f:
    credentials = json.load(f)

# Connection

In [5]:
client = ClickhouseClient(
    host = credentials['host'],
    port = credentials['port'],
    user = credentials['user'],
    password = credentials['password'],
    database = credentials['database']
)

In [7]:
client.execute('SHOW DATABASES')

[array(['_temporary_and_external_tables'], dtype=object),
 array(['default'], dtype=object),
 array(['raw'], dtype=object),
 array(['system'], dtype=object)]

# Database
## Create database

In [9]:
client.create_database('test')

In [10]:
client.execute('SHOW DATABASES')

[array(['_temporary_and_external_tables'], dtype=object),
 array(['default'], dtype=object),
 array(['raw'], dtype=object),
 array(['system'], dtype=object),
 array(['test'], dtype=object)]

## Drop database

In [11]:
client.drop_database('test')

In [12]:
client.execute('SHOW DATABASES')

[array(['_temporary_and_external_tables'], dtype=object),
 array(['default'], dtype=object),
 array(['raw'], dtype=object),
 array(['system'], dtype=object)]

# Table
## Create table

In [13]:
client.create_database('test')

In [24]:
fields = [
	{'name': 'meal_id', 'type': 'Int64'}, 
	{'name': 'category', 'type': 'String'},
	{'name': 'cuisine', 'type': 'String'}
]

In [25]:
client.create_table('meal', 'test', fields)

In [26]:
client.execute('SHOW TABLES IN test')

[array(['meal'], dtype=object)]

## Insert dataframe into table

In [27]:
df = pd.read_csv('../api/data/raw/meal_info.csv')

In [28]:
df.head()

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [29]:
client.insert_dataframe_into_table(table_name = 'meal', database = 'raw', df = df)

In [30]:
client.query_dataframe('SELECT * FROM raw.meal LIMIT 10')

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian
5,1311,Extras,Thai
6,1062,Beverages,Italian
7,1778,Beverages,Italian
8,1803,Extras,Thai
9,1198,Extras,Thai


## Drop table

In [31]:
client.execute('SHOW TABLES IN test')

[array(['meal'], dtype=object)]

In [32]:
client.drop_table('meal', 'test')

In [33]:
client.execute('SHOW TABLES IN test')

[]

In [34]:
client.drop_database('test')